In [1]:
import numpy as np
import cv2
import time
import matplotlib.pyplot as plt
from skimage.feature import hog, local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.datasets import mnist
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [3]:
# Normalize the images
X_train = X_train / 255.0
X_test = X_test / 255.0

In [4]:
# Resize images for deep learning models
X_train_resized = np.array([cv2.resize(img, (32, 32)) for img in X_train])
X_test_resized = np.array([cv2.resize(img, (32, 32)) for img in X_test])
X_train_resized = np.expand_dims(X_train_resized, axis=-1)
X_test_resized = np.expand_dims(X_test_resized, axis=-1)

In [5]:
# Feature extraction functions
datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1)

def extract_hog_features(images):
    return np.array([hog(img, pixels_per_cell=(4, 4), cells_per_block=(2, 2), feature_vector=True) for img in images])

def extract_lbp_features(images):
    return np.array([local_binary_pattern(img, P=8, R=1).flatten() for img in images])

def extract_edge_features(images):
    return np.array([cv2.Canny((img * 255).astype(np.uint8), 100, 200).flatten() for img in images])

In [6]:
# HOG + SVM Model
hog_features_train = extract_hog_features(X_train)
hog_features_test = extract_hog_features(X_test)
scaler = StandardScaler()
X_train_hog_scaled = scaler.fit_transform(hog_features_train)
X_test_hog_scaled = scaler.transform(hog_features_test)
hog_clf = SVC(kernel='rbf', C=10, gamma='scale')
hog_clf.fit(X_train_hog_scaled, y_train)
y_pred_hog = hog_clf.predict(X_test_hog_scaled)
hog_accuracy = accuracy_score(y_test, y_pred_hog)
hog_report = classification_report(y_test, y_pred_hog)

In [8]:
# ResNet50 + MLP Model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
feature_extractor = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)
X_train_resnet = feature_extractor.predict(np.repeat(X_train_resized, 3, axis=-1))
X_test_resnet = feature_extractor.predict(np.repeat(X_test_resized, 3, axis=-1))
X_train_resnet = X_train_resnet.reshape(X_train_resnet.shape[0], -1)
X_test_resnet = X_test_resnet.reshape(X_test_resnet.shape[0], -1)
mlp_model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train_resnet.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dense(10, activation='softmax')
])
mlp_model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
mlp_model.fit(X_train_resnet, y_train, epochs=10, batch_size=128, validation_data=(X_test_resnet, y_test))
resnet_accuracy = mlp_model.evaluate(X_test_resnet, y_test, verbose=0)[1]

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 281s 148ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 51s 164ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 16s 31ms/step - accuracy: 0.6051 - loss: 1.1882 - val_accuracy: 0.8667 - val_loss: 0.3994
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - accuracy: 0.8263 - loss: 0.5325 - val_accuracy: 0.9115 - val_loss: 0.2955
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 19s 31ms/step - accuracy: 0.8410 - loss: 0.4947 - val_accuracy: 0.9103 - val_loss: 0.2923
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 14s 31ms/step - accuracy: 0.8323 - loss: 0.5213 - val_accuracy: 0.9167 - val_loss: 0.2749
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - accuracy: 0.8323 - loss: 0.5112 - val_accuracy: 0.9164 - val_loss: 0.2737
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 14s 31ms/step - accuracy: 0.8378 - loss: 0.4961 - val_accuracy: 0.9245 - val_loss: 0.2401
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - accuracy: 0.8514 - loss: 0.4594 - val_accuracy: 0.9234 - val_loss: 0.2397
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 14s 30ms/step - accuracy: 0.8538 - loss: 0.4449 - 

In [9]:
# Print results
print("\nHOG + SVM Accuracy:", accuracy_score(y_test, y_pred_hog))
print("\nFine-Tuned ResNet50 + MLP Accuracy:", resnet_accuracy)


HOG + SVM Accuracy: 0.9826

Fine-Tuned ResNet50 + MLP Accuracy: 0.9302999973297119
